In [1]:
import clips
import sys
sys.path.append('./src/')
from clips_util import print_facts, print_rules, print_templates, build_read_assert

In [2]:
# create the CLIPS environment
env = clips.Environment()

import logging
logging.basicConfig(level=10, format='%(message)s')
router = clips.LoggingRouter()
env.add_router(router)

In [3]:
# Define templates
DEFTEMPLATE_PATIENT = """
(deftemplate patient 
    (slot name_is (type STRING)) 
    (slot age_is (type INTEGER))
    (slot gender_is (type SYMBOL)
        (allowed-symbols male female))
    (slot LDL_is (type FLOAT)) 
    (slot HDL_is (type FLOAT))
    (slot Lp_a_is (type FLOAT))
    (slot blood_pressure_is (type FLOAT))
    (slot smoking_status_is (type SYMBOL)
        (allowed-symbols yes no))
    (slot family_history_is (type SYMBOL)
        (allowed-symbols yes no))
    (slot triglycerides_is (type FLOAT))
    (slot overall_risk (type SYMBOL)
        (default none)
        (allowed-symbols none low moderate high))
)
"""
env.build(DEFTEMPLATE_PATIENT)

In [4]:
DEFTEMPLATE_RISK_INDICATOR = """
(deftemplate risk_indicator
    (slot type (type SYMBOL))
    (slot patient_name (type STRING))
)
"""
env.build(DEFTEMPLATE_RISK_INDICATOR)

In [5]:
#First layer
#Define rules
DEFRULE_HIGH_LDL = """
(defrule high_ldl_risk "High LDL cholesterol risk"
    ?p <- (patient (name_is ?name) (LDL_is ?ldl))
    (test (>= ?ldl 190))
    =>
    (assert (risk_indicator (type high_ldl) (patient_name ?name))))
"""
env.build(DEFRULE_HIGH_LDL)

DEFRULE_HIGH_BP = """
(defrule high_bp_risk "High blood pressure risk"
    ?p <- (patient (name_is ?name) (blood_pressure_is ?bp))
    (test (>= ?bp 140))
    =>
    (assert (risk_indicator (type high_bp) (patient_name ?name))))
"""
env.build(DEFRULE_HIGH_BP)

DEFRULE_HIGH_LPA = """
(defrule high_lpa_risk "Elevated Lp(a) levels risk"
    ?p <- (patient (name_is ?name) (Lp_a_is ?lpa))
    (test (>= ?lpa 30))
    =>
    (assert (risk_indicator (type high_lpa) (patient_name ?name))))
"""
env.build(DEFRULE_HIGH_LPA)

DEFRULE_SMOKER = """
(defrule smoker_risk "Smoking-related risk"
    ?p <- (patient (name_is ?name) (smoking_status_is yes))
    =>
    (assert (risk_indicator (type smoker) (patient_name ?name))))
"""
env.build(DEFRULE_SMOKER)

DEFRULE_HIGH_TRIGLYCERIDES = """
(defrule high_triglycerides_risk "High triglycerides risk"
    ?p <- (patient (name_is ?name) (triglycerides_is ?tg))
    (test (>= ?tg 150))
    =>
    (assert (risk_indicator (type high_triglycerides) (patient_name ?name))))
"""
env.build(DEFRULE_HIGH_TRIGLYCERIDES)

DEFRULE_FAMILY_HISTORY = """
(defrule family_history_risk "Family history risk"
    ?p <- (patient (name_is ?name) (family_history_is yes))
    =>
    (assert (risk_indicator (type family_history) (patient_name ?name))))
"""
env.build(DEFRULE_FAMILY_HISTORY)

#High HDL generally is good
DEFRULE_PROTECTIVE_HDL = """
(defrule protective_hdl "High HDL is generally protective"
    (patient (name_is ?name) (HDL_is ?hdl))
    (test (>= ?hdl 60))
    =>
    (assert (risk_indicator (type protective_hdl) (patient_name ?name))))
"""
env.build(DEFRULE_PROTECTIVE_HDL)

DEFRULE_HIGH_RISK_MALE_65_SMOKER = """
(defrule high_risk_male_65_smoker "Men >65 who have ever smoked are at higher risk"
    (patient (name_is ?name) (age_is ?age&:(> ?age 65)) (gender_is male) (smoking_status_is yes))
    =>
    (assert (risk_indicator (type male_65_smoker) (patient_name ?name))))
"""
env.build(DEFRULE_HIGH_RISK_MALE_65_SMOKER)

In [6]:
#Second layer
# If we detect any high-risk indicator, set the patient's overall_risk to high (unless protective HDL modifies it)
# We can refine logic: if multiple high-risk factors, high is certain; if protective HDL is present, maybe moderate.

DEFRULE_EVALUATE_RISK = """
(defrule evaluate_risk_factors
    ?p <- (patient (name_is ?name) (overall_risk none))
    (risk_indicator (patient_name ?name) (type ?type&~protective_hdl))
    =>
    (retract ?p)
    (assert (patient (name_is ?name) (overall_risk high)))
)
"""
env.build(DEFRULE_EVALUATE_RISK)

# If no high-risk factors found but protective HDL present, classify as low risk
DEFRULE_EVALUATE_LOW_RISK = """
(defrule evaluate_low_risk
    ?p <- (patient (name_is ?name) (overall_risk none))
    (not (risk_indicator (patient_name ?name) (type ~protective_hdl)))
    =>
    (retract ?p)
    (assert (patient (name_is ?name) (overall_risk low)))
)
"""
env.build(DEFRULE_EVALUATE_LOW_RISK)

# If high risk and protective HDL present, consider adjusting risk to moderate
DEFRULE_MODERATE_RISK = """
(defrule moderate_risk_adjustment
    ?p <- (patient (name_is ?name) (overall_risk high))
    (risk_indicator (patient_name ?name) (type protective_hdl))
    =>
    (retract ?p)
    (assert (patient (name_is ?name) (overall_risk moderate)))
)
"""
env.build(DEFRULE_MODERATE_RISK)

In [7]:
DEFRULE_PROVIDE_RECOMMENDATIONS = """
(defrule provide_recommendations "Provide recommendations"
    (patient (name_is ?name) (LDL_is ?ldl) (blood_pressure_is ?bp) (smoking_status_is ?smoker) 
             (family_history_is ?history) (triglycerides_is ?tg) (HDL_is ?hdl) (overall_risk ?risk))
    =>
    (printout t crlf "------- RECOMMENDATIONS FOR PATIENT " ?name "----------" crlf)
    (printout t "Overall risk classification: " ?risk crlf)

    (if (>= ?ldl 190) then 
        (printout t "- Consider consulting a healthcare provider about lipid-lowering medications." crlf))
    (if (>= ?bp 140) then 
        (printout t "- Adopt a healthier diet, regular exercise routine, and consider antihypertensive options." crlf))
    (if (eq ?smoker yes) then 
        (printout t "- Quit smoking to reduce cardiovascular risks." crlf))
    (if (eq ?history yes) then 
        (printout t "- Family history suggests closer monitoring and possibly earlier interventions." crlf))
    (if (>= ?tg 150) then 
        (printout t "- Consider dietary changes to lower triglycerides (reduce sugar, alcohol, and refined carbs)." crlf))
    (if (< ?hdl 40) then
        (printout t "- Low HDL (<40 mg/dL): consider lifestyle changes to improve HDL." crlf))
    (if (>= ?hdl 60) then
        (printout t "- Your HDL is high, which is protective. Continue with a healthy lifestyle." crlf))
    (if (and (eq ?smoker yes) (> (eval (str-cat "" ?bp)) 65) (eq ?risk high))
        then (printout t "- Note: Men over 65 who have smoked have increased atherosclerosis risk." crlf))
)
"""
env.build(DEFRULE_PROVIDE_RECOMMENDATIONS)

In [8]:
def get_valid_input(prompt, validation_function=None, error_message="Invalid input.", convert_to_lower=True):
    while True:
        user_input = input(prompt)
        if convert_to_lower and isinstance(user_input, str):
            user_input = user_input.lower().strip()
        if validation_function is None or validation_function(user_input):
            return user_input
        else:
            print(error_message)

def is_valid_gender(g):
    return g in ["male", "female"]

def is_valid_yes_no(y):
    return y in ["yes", "no"]

def is_valid_float(f):
    try:
        float(f)
        return True
    except ValueError:
        return False

def is_valid_int(i):
    try:
        int(i)
        return True
    except ValueError:
        return False

In [9]:
def input_data():
    name = input("Enter patient name: ")
    age = int(get_valid_input("Enter patient age (in years): ", is_valid_int, "Please enter a valid integer."))
    gender = get_valid_input("Enter patient gender (male or female): ", is_valid_gender, "Gender must be 'male' or 'female'.")
    ldl = float(get_valid_input("Enter LDL cholesterol level (mg/dL): ", is_valid_float))
    hdl = float(get_valid_input("Enter HDL cholesterol level (mg/dL): ", is_valid_float))
    lpa = float(get_valid_input("Enter Lp(a) level (mg/dL): ", is_valid_float))
    bp = float(get_valid_input("Enter blood pressure (mmHg): ", is_valid_float))
    smoking = get_valid_input("Is the patient a smoker? (yes or no): ", is_valid_yes_no, "Smoking status must be 'yes' or 'no'.")
    family_history = get_valid_input("Does the patient have a family history of CVD? (yes or no): ", is_valid_yes_no)
    triglycerides = float(get_valid_input("Enter triglycerides level (mg/dL): ", is_valid_float))

    return dict(
        name=name,
        age=age,
        gender=gender,
        ldl=ldl,
        hdl=hdl,
        lpa=lpa,
        bp=bp,
        smoking=smoking,
        family_history=family_history,
        triglycerides=triglycerides
    )

def assert_fact(env, patient_data):
    fact_str = f'(patient (name_is "{patient_data["name"]}") ' \
               f'(age_is {patient_data["age"]}) ' \
               f'(gender_is {patient_data["gender"]}) ' \
               f'(LDL_is {patient_data["ldl"]}) ' \
               f'(HDL_is {patient_data["hdl"]}) ' \
               f'(Lp_a_is {patient_data["lpa"]}) ' \
               f'(blood_pressure_is {patient_data["bp"]}) ' \
               f'(smoking_status_is {patient_data["smoking"]}) ' \
               f'(family_history_is {patient_data["family_history"]}) ' \
               f'(triglycerides_is {patient_data["triglycerides"]}) ' \
               f'(overall_risk none))'

    env.assert_string(fact_str)

In [10]:
def system():
    print("Welcome to the Atherosclerosis Risk Assessment Expert System.\n")
    while True:
        patient_data = input_data()
        env.reset()
        assert_fact(env, patient_data)
        env.run()

        # Check if user wants keep using
        another = get_valid_input("\nDo you want to evaluate another patient? (yes or no): ", is_valid_yes_no)
        if another == "yes":
            env.reset()
            system()
        if another == "no":
            print("Thanks for using the system. Goodbye!")
            break

system()


Welcome to the Atherosclerosis Risk Assessment Expert System.



Enter patient name:  Albert
Enter patient age (in years):  80
Enter patient gender (male or female):  male
Enter LDL cholesterol level (mg/dL):  200
Enter HDL cholesterol level (mg/dL):  200
Enter Lp(a) level (mg/dL):  2000
Enter blood pressure (mmHg):  200
Is the patient a smoker? (yes or no):  yes
Does the patient have a family history of CVD? (yes or no):  no
Enter triglycerides level (mg/dL):  200


------- RECOMMENDATIONS FOR PATIENT Albert----------
Overall risk classification: high
- Quit smoking to reduce cardiovascular risks.
- Family history suggests closer monitoring and possibly earlier interventions.
- Low HDL (<40 mg/dL): consider lifestyle changes to improve HDL.



Do you want to evaluate another patient? (yes or no):  no


Thanks for using the system. Goodbye!


In [11]:
print_facts(env)

(risk_indicator (type high_ldl) (patient_name "Albert"))
(patient (name_is "Albert") (age_is 0) (gender_is male) (LDL_is 0.0) (HDL_is 0.0) (Lp_a_is 0.0) (blood_pressure_is 0.0) (smoking_status_is yes) (family_history_is yes) (triglycerides_is 0.0) (overall_risk high))
(risk_indicator (type smoker) (patient_name "Albert"))
(risk_indicator (type family_history) (patient_name "Albert"))
Total facts: 4


SyntaxError: invalid syntax. Perhaps you forgot a comma? (463532008.py, line 1)